In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
df_guardian = pd.read_csv('/content/drive/My Drive/Text Project/Guardian all data.csv', header=None, encoding='cp1252')
df_guardian.columns = ['Comments']

In [7]:
df_guardian.head()

,Comments
0,Healthcare is a costly item in national budget...
1,Despite US legislation in 2010 that moved the ...
2,It is often said: “You get what you pay for.” ...
3,After decades of suffering some of the highest...
4,China: ‘A soaring demand for quality medical c...


In [8]:
# Import the Data Frame
import nltk
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('all')
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set( stopwords.words('english'))
from spacy.lang.en import English
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy 
import numpy as np
import pandas as pd

# Function to remove stopwords and replacing model names with brand names
# This function will be applied to all comments in our data frame

def filterReview(review):
    nlp = English()
    tokens = word_tokenize(review)
    word_str = ""

    # Removing numbers
    for w in tokens:
        try:
            float(w)
        
        # If not a number, we will consider the words
        except Exception as e:
            if w.lower() not in stop_words: 
                word_str = word_str + " " + w.lower()
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

# Backup fundtion to clean the data 
def removePunctuations(review):
    nlp = English()
    tokens = word_tokenize(review)
    stop_words_punct = set(stopwords.words('english') + list(punctuation))
    no_stopwords = [w.lower() for w in tokens if w.lower() not in stop_words_punct]
    word_str = ""
    for w in no_stopwords:
        # Removing numbers
        try:
            float(w)
        except Exception as e:
            word_str = word_str + " " + w
        continue
    
    # Lemmatization
    lemmatized = []
    doc = nlp(' '.join(word_str.split()))
    for word in doc:
        lemmatized.append(word.lemma_)
    
    return lemmatized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
df_guardian['Filter Comments NO punct'] = df_guardian['Comments'].map(removePunctuations)
# df_redUS['Filtered Comments'] = df_redUS['Comments'].map(filterReview)

In [10]:
df_guardian.head()

,Comments,Filter Comments NO punct
0,Healthcare is a costly item in national budget...,"[healthcare, costly, item, national, budget, g..."
1,Despite US legislation in 2010 that moved the ...,"[despite, us, legislation, move, country, clos..."
2,It is often said: “You get what you pay for.” ...,"[often, say, “, get, pay, for, ., ”, price, st..."
3,After decades of suffering some of the highest...,"[decade, suffer, high, maternal, child, mortal..."
4,China: ‘A soaring demand for quality medical c...,"[china, ‘, soar, demand, quality, medical, car..."


## Topic Modelling

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

In [0]:
dictionary_guardian = gensim.corpora.Dictionary(df_guardian['Filter Comments NO punct'])
dictionary_guardian.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_guardian.doc2bow(words) for words in df_guardian['Filter Comments NO punct']]
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=6, id2word=dictionary_guardian, passes=2, workers=2, chunksize=100, random_state=1000)

In [13]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_guardian.print_topics())

[   (   0,
        '0.010*"vaccine" + 0.006*"europe" + 0.006*"african" + 0.006*"eu" + '
        '0.006*"au" + 0.005*"." + 0.005*"confidence" + 0.005*"state" + '
        '0.005*"uk" + 0.004*"platform"'),
    (   1,
        '0.003*"woman" + 0.003*"nhs" + 0.003*"hospital" + 0.002*"care" + '
        '0.002*"ebola" + 0.002*"patient" + 0.002*"pay" + 0.002*"time" + '
        '0.002*"." + 0.002*"£"'),
    (   2,
        '0.011*"woman" + 0.009*"care" + 0.007*"spend" + 0.004*"man" + '
        '0.004*"hospital" + 0.004*"nhs" + 0.004*"many" + 0.004*"insurance" + '
        '0.004*"uk" + 0.004*"time"'),
    (   3,
        '0.013*"hospital" + 0.011*"patient" + 0.010*"pay" + 0.008*"private" + '
        '0.008*"care" + 0.007*"state" + 0.007*"insurance" + 0.007*"spend" + '
        '0.004*"cover" + 0.004*"treatment"'),
    (   4,
        '0.013*"ebola" + 0.008*"hospital" + 0.006*"die" + 0.005*"khan" + '
        '0.005*"patient" + 0.004*"adadevoh" + 0.004*"can" + 0.004*"liberia" + '
        '0.004*"worker

In [14]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_guardian.print_topics())

[   (   0,
        '0.011*"uk" + 0.011*"vaccine" + 0.006*"african" + 0.006*"eu" + '
        '0.006*"au" + 0.006*"europe" + 0.005*"confidence" + 0.005*"." + '
        '0.005*"state" + 0.005*"guardian"'),
    (   1,
        '0.018*"mali" + 0.009*"reform" + 0.007*"fee" + 0.007*"child" + '
        '0.007*"user" + 0.006*"rate" + 0.006*"mortality" + 0.005*"." + '
        '0.005*"also" + 0.005*"sow"'),
    (   2,
        '0.011*"care" + 0.011*"spend" + 0.005*"live" + 0.005*"time" + '
        '0.005*"many" + 0.005*"nhs" + 0.004*"hospital" + 0.004*"insurance" + '
        '0.004*"age" + 0.004*"medicare"'),
    (   3,
        '0.012*"hospital" + 0.010*"patient" + 0.009*"pay" + 0.008*"care" + '
        '0.007*"private" + 0.006*"woman" + 0.006*"nhs" + 0.006*"insurance" + '
        '0.006*"state" + 0.005*"spend"'),
    (   4,
        '0.013*"ebola" + 0.009*"£" + 0.008*"hospital" + 0.008*"drill" + '
        '0.006*"die" + 0.005*"khan" + 0.005*"use" + 0.005*"can" + '
        '0.004*"adadevoh" + 0.004*

## Topic Modelling after removing punctuations

In [0]:
dictionary_guardian = gensim.corpora.Dictionary(df_guardian['Filter Comments NO punct'])
dictionary_guardian.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_guardian.doc2bow(words) for words in df_guardian['Filter Comments NO punct']]
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=10, workers=2, chunksize=100, random_state=1000)

In [16]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_guardian.print_topics())

[   (   0,
        '0.011*"uk" + 0.011*"vaccine" + 0.006*"african" + 0.006*"eu" + '
        '0.006*"au" + 0.006*"europe" + 0.005*"confidence" + 0.005*"." + '
        '0.005*"state" + 0.005*"guardian"'),
    (   1,
        '0.018*"mali" + 0.009*"reform" + 0.007*"fee" + 0.007*"child" + '
        '0.007*"user" + 0.006*"rate" + 0.006*"mortality" + 0.005*"." + '
        '0.005*"also" + 0.005*"sow"'),
    (   2,
        '0.011*"care" + 0.011*"spend" + 0.005*"live" + 0.005*"time" + '
        '0.005*"many" + 0.005*"nhs" + 0.004*"hospital" + 0.004*"insurance" + '
        '0.004*"age" + 0.004*"medicare"'),
    (   3,
        '0.012*"hospital" + 0.010*"patient" + 0.009*"pay" + 0.008*"care" + '
        '0.007*"private" + 0.006*"woman" + 0.006*"nhs" + 0.006*"insurance" + '
        '0.006*"state" + 0.005*"spend"'),
    (   4,
        '0.013*"ebola" + 0.009*"£" + 0.008*"hospital" + 0.008*"drill" + '
        '0.006*"die" + 0.005*"khan" + 0.005*"use" + 0.005*"can" + '
        '0.004*"adadevoh" + 0.004*

In [17]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=30, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_guardian.print_topics())

[   (   0,
        '0.011*"uk" + 0.010*"vaccine" + 0.006*"nhs" + 0.006*"europe" + '
        '0.006*"african" + 0.006*"eu" + 0.006*"au" + 0.005*"confidence" + '
        '0.005*"." + 0.005*"state"'),
    (   1,
        '0.018*"mali" + 0.009*"reform" + 0.007*"fee" + 0.007*"child" + '
        '0.007*"user" + 0.006*"rate" + 0.006*"mortality" + 0.005*"." + '
        '0.005*"also" + 0.005*"sow"'),
    (   2,
        '0.012*"care" + 0.012*"spend" + 0.006*"live" + 0.006*"time" + '
        '0.006*"many" + 0.005*"insurance" + 0.005*"hospital" + '
        '0.004*"medicare" + 0.004*"age" + 0.004*"\'s"'),
    (   3,
        '0.012*"hospital" + 0.010*"patient" + 0.009*"pay" + 0.008*"care" + '
        '0.007*"private" + 0.006*"woman" + 0.006*"nhs" + 0.006*"state" + '
        '0.006*"insurance" + 0.005*"spend"'),
    (   4,
        '0.013*"ebola" + 0.009*"£" + 0.008*"hospital" + 0.008*"drill" + '
        '0.006*"die" + 0.005*"khan" + 0.005*"use" + 0.005*"can" + '
        '0.005*"adadevoh" + 0.004*"nhs"

## Additional Stop Words removal

In [0]:
new_stopword_list = ['gt', '`', '-', '“', '-', '/', 'amp', '•', 'subreddit', '’', '\'s',"\'\'", "\'", '”', 'argument', 'amitheasshole', 'https', 'http']
def additionalStop(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list]

In [0]:
df_guardian['filter V1'] = df_guardian['Filter Comments NO punct'].map(additionalStop)

In [0]:
dictionary_guardian = gensim.corpora.Dictionary(df_guardian['filter V1'])
dictionary_guardian.filter_extremes(no_below=1, no_above=0.7) #, keep_n=100000)
bow_corpus_before = [dictionary_guardian.doc2bow(words) for words in df_guardian['filter V1']]

In [0]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=10, workers=2, chunksize=100, random_state=1000)

In [22]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=3, id2word=dictionary_guardian, passes=10, workers=2, chunksize=100, random_state=1000)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(lda_model_guardian.print_topics())

[   (   0,
        '0.010*"hospital" + 0.008*"patient" + 0.007*"woman" + 0.007*"care" + '
        '0.007*"private" + 0.006*"spend" + 0.006*"state" + 0.006*"insurance" + '
        '0.004*"free" + 0.004*"cover"'),
    (   1,
        '0.008*"hospital" + 0.008*"care" + 0.007*"ebola" + 0.005*"time" + '
        '0.005*"spend" + 0.004*"drill" + 0.004*"patient" + 0.003*"centre" + '
        '0.003*"drug" + 0.003*"worker"'),
    (   2,
        '0.011*"vaccine" + 0.006*"europe" + 0.006*"african" + '
        '0.006*"confidence" + 0.005*"state" + 0.005*"platform" + '
        '0.004*"global" + 0.004*"science" + 0.003*"link" + 0.003*"european"')]


In [23]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=10, workers=2, chunksize=100, random_state=1000)
lda_model_guardian.print_topics()

[(0,
  '0.027*"woman" + 0.008*"likely" + 0.007*"pain" + 0.006*"tell" + 0.005*"poor" + 0.005*"household" + 0.004*"spend" + 0.004*"think" + 0.004*"poverty" + 0.004*"publish"'),
 (1,
  '0.011*"care" + 0.010*"drill" + 0.007*"hospital" + 0.007*"spend" + 0.005*"medicare" + 0.004*"time" + 0.004*"insurance" + 0.004*"many" + 0.004*"read" + 0.004*"innovation"'),
 (2,
  '0.020*"vaccine" + 0.010*"confidence" + 0.009*"europe" + 0.007*"global" + 0.007*"science" + 0.006*"show" + 0.006*"link" + 0.006*"think" + 0.006*"measle" + 0.006*"larson"'),
 (3,
  '0.010*"mali" + 0.007*"state" + 0.006*"african" + 0.005*"reform" + 0.005*"insurance" + 0.004*"child" + 0.004*"spend" + 0.004*"trump" + 0.004*"provide" + 0.004*"universal"'),
 (4,
  '0.014*"hospital" + 0.011*"patient" + 0.008*"private" + 0.008*"care" + 0.006*"spend" + 0.006*"ebola" + 0.005*"state" + 0.005*"treatment" + 0.005*"insurance" + 0.004*"time"')]

In [24]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_guardian.print_topics()

[(0,
  '0.028*"woman" + 0.008*"likely" + 0.007*"pain" + 0.006*"tell" + 0.005*"poor" + 0.005*"household" + 0.004*"poverty" + 0.004*"think" + 0.004*"female" + 0.004*"publish"'),
 (1,
  '0.011*"care" + 0.010*"drill" + 0.007*"hospital" + 0.007*"spend" + 0.005*"medicare" + 0.004*"read" + 0.004*"insurance" + 0.004*"innovation" + 0.004*"time" + 0.004*"many"'),
 (2,
  '0.020*"vaccine" + 0.010*"confidence" + 0.009*"europe" + 0.007*"global" + 0.007*"science" + 0.006*"think" + 0.006*"show" + 0.006*"link" + 0.006*"larson" + 0.006*"measle"'),
 (3,
  '0.011*"mali" + 0.007*"state" + 0.007*"african" + 0.005*"reform" + 0.005*"child" + 0.004*"trump" + 0.004*"provide" + 0.004*"spend" + 0.004*"universal" + 0.004*"free"'),
 (4,
  '0.014*"hospital" + 0.011*"patient" + 0.008*"private" + 0.008*"care" + 0.006*"spend" + 0.006*"state" + 0.006*"ebola" + 0.006*"insurance" + 0.005*"treatment" + 0.004*"time"')]

In [0]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, chunksize = 64, eval_every = 5, per_word_topics = 3, minimum_probability = 0.1, passes=25, random_state=23)

In [26]:
lda_model_guardian.print_topics()

[(0,
  '0.020*"woman" + 0.006*"african" + 0.006*"state" + 0.006*"likely" + 0.006*"pain" + 0.005*"tell" + 0.005*"centre" + 0.005*"platform" + 0.003*"head" + 0.003*"right"'),
 (1,
  '0.013*"hospital" + 0.011*"care" + 0.010*"patient" + 0.008*"spend" + 0.007*"private" + 0.007*"insurance" + 0.007*"state" + 0.005*"provide" + 0.004*"free" + 0.004*"many"'),
 (2,
  '0.000*"hospital" + 0.000*"care" + 0.000*"patient" + 0.000*"spend" + 0.000*"state" + 0.000*"insurance" + 0.000*"many" + 0.000*"private" + 0.000*"woman" + 0.000*"time"'),
 (3,
  '0.015*"ebola" + 0.009*"hospital" + 0.009*"drill" + 0.006*"khan" + 0.005*"adadevoh" + 0.005*"liberia" + 0.004*"centre" + 0.004*"patient" + 0.004*"worker" + 0.004*"nigeria"'),
 (4,
  '0.011*"vaccine" + 0.006*"europe" + 0.006*"confidence" + 0.005*"time" + 0.005*"england" + 0.005*"link" + 0.005*"spend" + 0.004*"global" + 0.004*"budget" + 0.004*"rate"')]

In [0]:
# df_redUS.to_csv('RedditUSA.csv')

## Preprocessing to remove the most common trivial words

In [0]:
from collections import Counter
s = df_guardian['filter V1']
text = s.apply(pd.Series).stack().reset_index(drop=True)
word_counts = Counter(text)
common_words = word_counts.most_common()

In [0]:
df = pd.DataFrame(common_words)
df.to_csv('/content/drive/My Drive/Text Project/df.csv')

In [0]:
common_stops = ['people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

In [0]:
new_stopword_list_v1 = ['//www.guardian.com', 'people','much','well','time', 'would', 'like', 'make', 'work','long','still','even','give', 'good', 'think', 'want', 'right', 'thing', 'year', 'take', 'also', 'know', 'come', 'every', 'life', 'look', 'have', 'will', 'tell', 'believe', 'talk', 'seem', 'since', 'show', 'else','like'] 

def additionalStop_V1(comment):
    return [w for w in comment if len(w)>3 and w not in new_stopword_list_v1]

df_guardian['filter V2'] = df_guardian['filter V1'].map(additionalStop_V1)

In [32]:
df_guardian.head()

,Comments,Filter Comments NO punct,filter V1,filter V2
0,Healthcare is a costly item in national budget...,"[healthcare, costly, item, national, budget, g...","[healthcare, costly, item, national, budget, w...","[healthcare, costly, item, national, budget, v..."
1,Despite US legislation in 2010 that moved the ...,"[despite, us, legislation, move, country, clos...","[despite, legislation, move, country, close, a...","[despite, legislation, move, country, close, a..."
2,It is often said: “You get what you pay for.” ...,"[often, say, “, get, pay, for, ., ”, price, st...","[often, price, stand, proxy, quality, whether,...","[often, price, stand, proxy, quality, whether,..."
3,After decades of suffering some of the highest...,"[decade, suffer, high, maternal, child, mortal...","[decade, suffer, high, maternal, child, mortal...","[decade, suffer, high, maternal, child, mortal..."
4,China: ‘A soaring demand for quality medical c...,"[china, ‘, soar, demand, quality, medical, car...","[china, soar, demand, quality, medical, care, ...","[china, soar, demand, quality, medical, care, ..."


In [0]:
dictionary_guardian = gensim.corpora.Dictionary(df_guardian['filter V2'])
dictionary_guardian.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)
bow_corpus_before = [dictionary_guardian.doc2bow(words) for words in df_guardian['filter V2']]

In [34]:
lda_model_guardian = gensim.models.LdaMulticore(bow_corpus_before, num_topics=5, id2word=dictionary_guardian, passes=50, workers=2, chunksize=100, random_state=1000)
lda_model_guardian.print_topics()

[(0,
  '0.013*"drill" + 0.006*"trump" + 0.005*"news" + 0.005*"innovation" + 0.005*"surgical" + 0.004*"guardian" + 0.004*"price" + 0.004*"mesh" + 0.004*"sterilise" + 0.004*"adapt"'),
 (1,
  '0.009*"insurance" + 0.005*"cover" + 0.005*"krona" + 0.005*"upfront" + 0.004*"emergency" + 0.004*"japan" + 0.004*"change" + 0.004*"african" + 0.004*"payment" + 0.004*"centre"'),
 (2,
  '0.015*"ebola" + 0.010*"vaccine" + 0.007*"khan" + 0.006*"europe" + 0.005*"confidence" + 0.005*"adadevoh" + 0.005*"disease" + 0.005*"liberia" + 0.004*"centre" + 0.004*"trust"'),
 (3,
  '0.006*"clinic" + 0.006*"problem" + 0.005*"insurance" + 0.005*"india" + 0.004*"germany" + 0.004*"sector" + 0.004*"half" + 0.004*"cover" + 0.004*"italy" + 0.004*"facility"'),
 (4,
  '0.022*"woman" + 0.012*"mali" + 0.006*"reform" + 0.006*"likely" + 0.006*"pain" + 0.005*"medicare" + 0.004*"insurance" + 0.004*"mortality" + 0.004*"user" + 0.004*"universal"')]

## Summarization

In [0]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df_guardian['Comments']:
  sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x] # flatten list

In [36]:
sentences[:5]

['Healthcare is a costly item in national budgets, but who gets the best value for money, and who the best outcomes?',
 'We compare the systems in some of the world’s leading countries for healthcare.',
 'France\nUpfront payments: yes\n\nData: France scores highly on GP numbers and on spending as a percentage of GDP, but it comes off badly in the Commonwealth Fund’s international comparison reports, ranking only ninth out of 11 in the latest assessment.',
 'Summary\n\nUnder France’s state-run equivalent of the NHS, the majority of patients must pay the doctor or practitioner upfront.',
 'The state then reimburses them in part or in full.']

In [37]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2019-12-01 02:48:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-01 02:48:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-01 02:48:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [0]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [0]:
# function to remove stopwords
def remove_stopwords_sentence(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [0]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords_sentence(r.split()) for r in clean_sentences]

In [0]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [0]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [0]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [0]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [47]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

How a poor community in Mali became a trailblazer for tracking child mortality
 Read more
“We’ve seen a number of countries where, when you take away user fees too quickly, you overload the system,” said Lejeune, adding that any health ministry reforms needed the full support of the presidency, which this government had.
I’m not making that up: huge swathes of the population never used the services because they just couldn’t afford them.”

Charlotte Lejeune, Mali country director for the Clinton Health Access Initiative, which has been supporting the health ministry’s reforms, said the success over the next four years would depend on how progressively the government rolled out the changes.
Gershlick says a 4% budget hike would enable the NHS to improve the number of mental health patients getting treatment from four in 10 to seven in 10, boost capital spending, reduce waiting times and, crucially, do more in the field of prevention to stop people needing the NHS in the first place.
“It